In [ ]:
# Google Colabでkaggle.jsonをアップロード
from google.colab import files
files.upload()

# Google Colabでkaggle.jsonをフォルダ移動
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Google Colabでデータセットをダウンロード
!pip install kaggle
!kaggle competitions download -c m5-forecasting-uncertainty